In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# uncomment to disable numba
# os.environ['NOJIT'] = 'true'

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import pprint
from njit_multisymbol import *
from procedures import utc_ms, load_live_config, fetch_market_specific_settings, make_get_filepath
from pure_funcs import (
    date_to_ts2,
    ts_to_date_utc,
    tuplify,
    numpyize,
    stats_multi_to_df,
    fills_multi_to_df,
    calc_drawdowns,
    calc_sharpe_ratio,
    analyze_fills_multi,
    live_config_dict_to_list_recursive_grid,
)
from njit_funcs import round_dynamic
from plotting import plot_fills_multi, plot_pnls_long_short, plot_pnls_separate, plot_pnls_stuck
from numba import njit
from downloader import load_hlc_cache, prepare_multsymbol_data
from backtest_multi import prep_hlcs_mss_config, prep_config_multi, args2config

In [ ]:
plt.rcParams["figure.figsize"] = [24, 13.5]
plt.rcParams["figure.facecolor"] = "w"
pd.set_option("display.precision", 10)

### multi symbol backtest with lossless auto unstuck

- if a position is stuck, bot will use profits made on other markets to realize losses for the stuck position
- if multiple positions are stuck, select the stuck pos whose price action distance is the lowest
- each live config's individual auto unstuck is disabled

In [ ]:
cfg = {
    "global": {
        "TWE_long": 1.9628217193346074,
        "TWE_short": 8.873613532492868,
        "loss_allowance_pct": 0.007057868173311633,
        "stuck_threshold": 0.8214753608753934,
        "unstuck_close_pct": 0.0011074274412150183,
    },
    "long": {
        "ddown_factor": 1.3552241067565591,
        "ema_span_0": 1251.5875524266064,
        "ema_span_1": 604.1687419843032,
        "enabled": True,
        "initial_eprice_ema_dist": -0.020367702184693325,
        "initial_qty_pct": 0.013372380921445446,
        "markup_range": 0.003390533528629384,
        "min_markup": 0.005905020194249954,
        "n_close_orders": 2.1233372580530903,
        "rentry_pprice_dist": 0.049046697381191565,
        "rentry_pprice_dist_wallet_exposure_weighting": 0.5612700282178685,
        "wallet_exposure_limit": 0.39256434386692146,
    },
    "short": {
        "ddown_factor": 0.3907245872883093,
        "ema_span_0": 1103.2107410642989,
        "ema_span_1": 1370.6087776402064,
        "enabled": False,
        "initial_eprice_ema_dist": 0.0017511634241226537,
        "initial_qty_pct": 0.044680299514502905,
        "markup_range": 0.0017555611262703857,
        "min_markup": 0.007697306221784713,
        "n_close_orders": 12.949796081895949,
        "rentry_pprice_dist": 0.02703134258902625,
        "rentry_pprice_dist_wallet_exposure_weighting": 2.4981211753897754,
        "wallet_exposure_limit": 1.7747227064985736,
    },
}
starting_balance = 1000000
symbols = ["BALUSDT", "BANDUSDT", "CELRUSDT", "OMGUSDT", "SKLUSDT"]

In [ ]:
class Args:
    def __init__(self):
        self.symbols = symbols
        self.symbols = {s: "" for s in self.symbols}
        self.user = "binance_01"
        self.start_date = "2021-05-01"
        self.end_date = "now"
        self.starting_balance = starting_balance
        self.long_enabled = cfg["long"]["enabled"]
        self.short_enabled = cfg["short"]["enabled"]
        self.TWE_long = cfg["global"]["TWE_long"]
        self.TWE_short = cfg["global"]["TWE_short"]


args = Args()
config = args2config(args)
config["base_dir"] = "backtests"
config["symbols"] = tuple(sorted(set(config["symbols"])))
hlcs, mss, config = await prep_hlcs_mss_config(config)

In [ ]:
symbols = tuple(sorted(set(config["symbols"])))

# specify live configs for each symbol. Use a single live config, or load separately for each symbol.
# live_configs = {symbol: load_live_config(f"configs/live/multisymbol/no_AU/{symbol}.json") for symbol in symbols}
live_configs = {symbol: cfg for symbol in symbols}
for s in live_configs:
    live_configs[s]["long"]["wallet_exposure_limit"] = cfg["global"]["TWE_long"] / len(symbols)
    live_configs[s]["short"]["wallet_exposure_limit"] = cfg["global"]["TWE_short"] / len(symbols)
live_configs_np = numpyize(
    [live_config_dict_to_list_recursive_grid(live_configs[s]) for s in symbols]
)

do_longs = tuplify([cfg["long"]["enabled"] for s in config["symbols"]])
do_shorts = tuplify([cfg["short"]["enabled"] for s in config["symbols"]])
qty_steps = tuplify([mss[symbol]["qty_step"] for symbol in config["symbols"]])
price_steps = tuplify([mss[symbol]["price_step"] for symbol in config["symbols"]])
min_costs = tuplify([mss[symbol]["min_cost"] for symbol in config["symbols"]])
min_qtys = tuplify([mss[symbol]["min_qty"] for symbol in config["symbols"]])
c_mults = tuplify([mss[symbol]["c_mult"] for symbol in config["symbols"]])
maker_fee = next(iter(mss.values()))["maker"]
starting_balance = config["starting_balance"]

In [ ]:
hlcs_clipped = hlcs  # [:,0:60000] # to backtest on subset on data
# will compile JIT on the first run, then be faster on subsequent runs
sts = utc_ms()
res = backtest_multisymbol_recursive_grid(
    hlcs_clipped,
    starting_balance,
    maker_fee,
    do_longs,
    do_shorts,
    c_mults,
    symbols,
    qty_steps,
    price_steps,
    min_costs,
    min_qtys,
    live_configs_np,
    cfg["global"]["loss_allowance_pct"],
    cfg["global"]["stuck_threshold"],
    cfg["global"]["unstuck_close_pct"],
)
print(f"time elapsed for backtest {(utc_ms() - sts) / 1000:.6f}s")
fills, stats = res

In [ ]:
sts = utc_ms()
fdf = fills_multi_to_df(fills, symbols, c_mults)
sdf = stats_multi_to_df(stats, symbols, c_mults)
print(f"time elapsed for analysis {(utc_ms() - sts) / 1000:.6f}s")

In [ ]:
params = {"TWE_long": cfg["global"]["TWE_long"], "TWE_short": cfg["global"]["TWE_short"]}
params = cfg["global"]
analysis = analyze_fills_multi(sdf, fdf, params)

In [ ]:
mkl = max([len(k) for k in analysis])
for k, v in analysis.items():
    if isinstance(v, dict):
        continue
        mkls = max([len(s) for s in v])
        for symbol in v:
            mkl1 = max([len(k) for k in v[symbol]])
            for k1, v1 in v[symbol].items():
                print(f"    {symbol: <{mkls}} {k1: <{mkl1}} {round_dynamic(v1, 6)}")
            print()
    else:
        print(f"{k: <{mkl}} {round_dynamic(v, 6)}")
adf = pd.DataFrame({k: v for k, v in analysis["individual_analyses"].items()})
adf

In [ ]:
if not adf.T.upnl_pct_min_long.isna().all():
    print("upnl pct min long")
    print(adf.T.upnl_pct_min_long.sort_values())
    print()
if not adf.T.upnl_pct_min_short.isna().all():
    print("upnl pct min short")
    print(adf.T.upnl_pct_min_short.sort_values())

In [ ]:
if not (adf.T.loss_profit_ratio_long == 1.0).all():
    print("loss_profit_ratio_long")
    print(adf.T.loss_profit_ratio_long.sort_values(ascending=False))
    print()
if not (adf.T.loss_profit_ratio_short == 1.0).all():
    print("loss_profit_ratio_short")
    print(adf.T.loss_profit_ratio_short.sort_values(ascending=False))

In [ ]:
adf.T.pnl_ratio.sort_values()

In [ ]:
# find worst performers
worsts = []
clip = 0.4
for x in [
    adf.T.pnl_ratio.sort_values(),
    # adf.T.loss_profit_ratio_short.sort_values(ascending=False),
    adf.T.loss_profit_ratio_long.sort_values(ascending=False),
    # adf.T.upnl_pct_min_short.sort_values(),
    # adf.T.upnl_pct_min_long.sort_values(),
]:
    worsts.append(list(dict(x.iloc[: int(len(x) * clip)])))
to_drop = [x for x in worsts[0] if all([x in w for w in worsts])]
to_drop

In [ ]:
sdf

In [ ]:
fdf

In [ ]:
# plot drawdowns
min_multiplier = 60 * 24
drawdowns = calc_drawdowns(sdf.equity)
drawdowns_daily = drawdowns.groupby(drawdowns.index // min_multiplier * min_multiplier).min()
drawdowns_ten_worst = drawdowns_daily.sort_values().iloc[:10]
print(drawdowns_ten_worst)
drawdowns_ten_worst.plot(style="ro")
drawdowns.plot()

In [ ]:
plot_pnls_stuck(sdf, fdf, start_pct=0.0, end_pct=1.0)

In [ ]:
plot_pnls_separate(sdf, fdf)

In [ ]:
plot_pnls_long_short(sdf, fdf)

In [ ]:
# inspect two months before and two months after location of worst drawdown
drawdowns = calc_drawdowns(sdf.equity)
worst_drawdown_loc = drawdowns.sort_values().iloc[:1].index[0]
wdls = worst_drawdown_loc - 60 * 24 * 30 * 2
wdle = worst_drawdown_loc + 60 * 24 * 30 * 2
sdfc = sdf.loc[wdls:wdle]
sdfc.balance.plot()
sdfc.equity.plot()

In [ ]:
# inspect for each symbol
for symbol in symbols:
    print(symbol)
    plot_fills_multi(symbol, sdf.loc[wdls:wdle], fdf.loc[wdls:wdle]).show()

In [ ]:
# inspect individual lowest drawdowns
upnl_pct_mins = sdf[[c for c in sdf.columns if "upnl" in c]].min().sort_values()
print(upnl_pct_mins)
print()
n = 60 * 24 * 60
upnl_pct_idxs = dict(sdf[[c for c in sdf.columns if "upnl" in c]].idxmin())
for sym in dict(upnl_pct_mins):
    idx = upnl_pct_idxs[sym]
    if np.isnan(idx):
        continue
    print(sym, idx)
    plot_fills_multi(
        sym[: sym.find("_")], sdf.loc[idx - n : idx + n], fdf.loc[idx - n : idx + n]
    ).show()

In [ ]:
# exposures
sdf[[c for c in sdf.columns if "WE" in c]].sum(axis=1).plot()

In [ ]:
# find worst realized losses for each symbol
wdds = {}
mins_before_and_after = 60 * 24 * 30 * 1
for symbol in symbols:
    wdd = calc_drawdowns(fdf[fdf.symbol == symbol].pnl.cumsum() + sdf.balance.iloc[0])
    wdds[symbol] = [abs(wdd.min()), wdd.idxmin()]
for symbol, wdd in sorted(wdds.items(), key=lambda x: x[1][0], reverse=True):
    print(symbol, f"pct loss: {wdd[0] * 100:.2f}% n_days: {wdd[1] / (60 * 24):.2f}")
    fdfc = fdf.loc[wdd[1] - mins_before_and_after : wdd[1] + mins_before_and_after]
    sdfc = sdf.loc[wdd[1] - mins_before_and_after : wdd[1] + mins_before_and_after]
    plot_fills_multi(symbol, sdfc, fdfc).show()